In [10]:
import sys  
import json
import pandas as pd
import numpy as np
import lightgbm as lgb

sys.path.insert(0, '../')
import gbdt_forecast

In [11]:
params_path = '../params/params_eem2020_wind_example.json'
with open(params_path, 'r', encoding='utf-8') as file:
    params_json = json.loads(file.read())

In [12]:
params_json

{'trial_name': 'trial0',
 'trial_comment': 'test',
 'path_result': './result/eem20/',
 'path_raw_data': './data/eem20/',
 'filename_raw_data': 'eem2020-raw.csv',
 'path_preprocessed_data': './data/eem20/preprocessed/',
 'filename_preprocessed_data': 'eem2020-preprocessed.csv',
 'data_resolution': '60min',
 'splits': {'train': [['2000-01-01 01:00:00', '2000-10-01 00:00:00']],
  'valid': [['2000-11-01 01:00:00', '2000-12-31 00:00:00']]},
 'sites': ['SE1', 'SE2', 'SE3', 'SE4'],
 'features': ['WindSpeed_1',
  'WindSpeed_2',
  'WindSpeed_3',
  'WindSpeed_4',
  'WindDirection_1',
  'WindDirection_2',
  'WindDirection_3',
  'WindDirection_4'],
 'target': 'Power',
 'diff_target_with_physical': False,
 'target_smoothing_window': 1,
 'regression_params': {'type': ['quantile'],
  'alpha_range': [0.01, 1.0, 0.01],
  'quantile_postprocess': 'sorting',
  'target_min_max': [0, 1000]},
 'model_params': {'lightgbm': {'learning_rate': 0.1,
   'num_trees': 20,
   'boosting': 'gbdt',
   'max_leaves': 32,


In [38]:
trial_name = 'trial0'
model = 'lightgbm'

sites = range(len(params_json['sites']))
splits = range(len(params_json['splits']['valid']))
alpha_q = np.arange(params_json['regression_params']['alpha_range'][0],
                    params_json['regression_params']['alpha_range'][1],
                    params_json['regression_params']['alpha_range'][2])

gbm_split = []
for split in splits:    
    gbm_site = []
    for site in sites:
        gbm_q = {}
        for alpha in alpha_q:
            file_name = '../result/eem20/'+trial_name+'/gbm_model/gbm_model_'+model+'_q_quantile{0:.2f}_split_{1}_site_{2}.txt'.format(alpha, split, site)
            if model == 'lightgbm': 
                gbm = lgb.Booster(model_file=file_name)
            elif model == 'catboost':
                gbm = cb.CatBoostRegressor().load_model(file_name)
            gbm_q['quantile{0:.2f}'.format(alpha)] = gbm
        gbm_site.append(gbm_q)
    gbm_split.append(gbm_site)

In [40]:
df_X = pd.read_csv('../data/eem20/preprocessed/eem2020-preprocessed.csv', index_col=[0,1], header=[0,1])

In [41]:
trial = gbdt_forecast.Trial(params_json)

In [65]:
dfs_y_pred = trial.predict_model_split_site([[df_X['SE1'], df_X['SE2'], df_X['SE3'], df_X['SE4']]], {model: gbm_split})

  0%|          | 0/4 [00:00<?, ?it/s]

Predicting...


100%|██████████| 4/4 [00:02<00:00,  1.94it/s]


In [72]:
df_pred = pd.concat(dfs_y_pred[model][0], axis=1)
df_pred.columns = [f'SE{site_id+1}_quantile{q_id+1}' for q_id in range(len(dfs_y_pred[model][0][0].columns)) for site_id in sites]
df_pred.droplevel(level=0, axis=0)

In [75]:
df_pred.to_csv("./submission-0.csv", header=True)

In [76]:
df_pred.head()

SE1_quantile1  SE2_quantile1  SE3_quantile1  \
ref_datetime valid_datetime                                                     
2000-01-02   2000-01-02 00:00:00       2.625833       8.032666      10.299038   
             2000-01-02 01:00:00       2.625833       7.494488      10.299038   
             2000-01-02 02:00:00       2.625833       8.032666      10.546276   
             2000-01-02 03:00:00       2.625833       8.032666      10.546276   
             2000-01-02 04:00:00       2.625833       7.494488      10.546276   

                                  SE4_quantile1  SE1_quantile2  SE2_quantile2  \
ref_datetime valid_datetime                                                     
2000-01-02   2000-01-02 00:00:00      13.524069      15.488036      17.436348   
             2000-01-02 01:00:00      13.524069      15.488036      17.849544   
             2000-01-02 02:00:00      13.524069      15.457736      18.021248   
             2000-01-02 03:00:00      13.657336      15.116212      17.658164   
             2000-01-02 04:00:00      13.657336      15.116212      17.283963   

                                  SE3_quantile2  SE4_quantile2  SE1_quantile3  \
ref_datetime valid_datetime                                                     
2000-01-02   2000-01-02 00:00:00      19.230108      20.182856      22.811934   
             2000-01-02 01:00:00      19.230108      20.182856      21.949742   
             2000-01-02 02:00:00      19.230108      20.407110      22.678376   
             2000-01-02 03:00:00      19.041101      21.853178      22.524370   
             2000-01-02 04:00:00      19.041101      22.028306      22.207718   

                                  SE2_quantile3  ...  SE3_quantile97  \
ref_datetime valid_datetime                      ...                   
2000-01-02   2000-01-02 00:00:00      25.310804  ...     1000.000000   
             2000-01-02 01:00:00      22.811934  ...     1000.000000   
             2000-01-02 02:00:00      22.811934  ...      976.472435   
             2000-01-02 03:00:00      22.737912  ...      949.377620   
             2000-01-02 04:00:00      22.292007  ...      966.215947   

                                  SE4_quantile97  SE1_quantile98  \
ref_datetime valid_datetime                                        
2000-01-02   2000-01-02 00:00:00     1000.000000     1000.000000   
             2000-01-02 01:00:00     1000.000000     1000.000000   
             2000-01-02 02:00:00     1000.000000     1000.000000   
             2000-01-02 03:00:00      968.851326      972.809635   
             2000-01-02 04:00:00      972.412788      992.959460   

                                  SE2_quantile98  SE3_quantile98  \
ref_datetime valid_datetime                                        
2000-01-02   2000-01-02 00:00:00     1000.000000          1000.0   
             2000-01-02 01:00:00     1000.000000          1000.0   
             2000-01-02 02:00:00     1000.000000          1000.0   
             2000-01-02 03:00:00      993.581721          1000.0   
             2000-01-02 04:00:00     1000.000000          1000.0   

                                  SE4_quantile98  SE1_quantile99  \
ref_datetime valid_datetime                                        
2000-01-02   2000-01-02 00:00:00          1000.0          1000.0   
             2000-01-02 01:00:00          1000.0          1000.0   
             2000-01-02 02:00:00          1000.0          1000.0   
             2000-01-02 03:00:00          1000.0          1000.0   
             2000-01-02 04:00:00          1000.0          1000.0   

                                  SE2_quantile99  SE3_quantile99  \
ref_datetime valid_datetime                                        
2000-01-02   2000-01-02 00:00:00          1000.0          1000.0   
             2000-01-02 01:00:00          1000.0          1000.0   
             2000-01-02 02:00:00          1000.0          1000.0   
             2000-01-02 03:00:00          1000.0      